In [21]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel
import re



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/"
upload_targ = "/DATA/MYRISTATE/DATA/1_week/"

In [12]:
test_name = glob(directory_targ + '*')[0]
imgs_address = test_name
print(test_name)

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230307_plate010.csv


In [63]:
plate_list = glob(directory_targ + "*_Plate*/*/")
datadict = {'address_total' : plate_list}
dataframe = pd.DataFrame(data=datadict)

# print(dataframe['address_total'][0])

dataframe['video_name'] = [address[:-1].split('/')[-1] for address in dataframe['address_total']]
dataframe['plate_nr'] = [re.split('_|/', address.split('_')[-1])[0][5:] for address in dataframe['address_total']]
dataframe['video_nr'] = [int(re.split('_|/', address[:-1])[-1]) for address in dataframe['address_total']]
dataframe['date_imaged'] = [address.split('/')[-1][:8] for address in dataframe['address_total']]

parent_folder = []
data_table = []
magnification = []
fps = []
mode = []

for row in dataframe.iloc:
    print(row['address_total'])
    parent_folder.append(str(Path(row['address_total']).parent))
    excel_file = glob(str(Path(row['address_total']).parent) + f'/*{row["date_imaged"]}*{row["plate_nr"]}.xl*')
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
#         print(row['video_name'])
#         print(excel_table['Unnamed: 0'][21])
#         print(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'])
        magnification.append(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'].iloc[0])
        fps.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['FPS'].iloc[0])
        mode.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Bright-field (BF)\nor\nFluorescence (F)'].iloc[0])
        
    else:
        csv_file = glob(str(Path(row['address_total']).parent) + f'/{row["date_imaged"]}*{row["plate_nr"]}.csv')
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1,sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1]>df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")
                
#             print(csv_table['video'])
#             print(row['video_name'])
            magnification.append(csv_table[csv_table['video']==int(row['video_name'])]['Lens'].iloc[0])
            fps.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['fps'].iloc[0])
            mode.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['Illumination'].iloc[0])

        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe['fps'] = fps
dataframe['magnification'] = magnification
dataframe['mode'] = mode
dataframe['parent_folder'] = parent_folder
dataframe['data_table'] = data_table

dataframe = dataframe.sort_values(by='video_name')
# dataframe = dataframe[dataframe['video_nr']== 5]
# dataframe = dataframe[dataframe['date_imaged'] == '20221026']
# print(len(dataframe))


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method='pad', limit_direction='forward')
if dataframe.isnull().values.any():
    raise("This excel sheet is unworkable, please ask the responsible person")
dataframe

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate051/002/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate051/001/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/014/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/006/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/002/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/007/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/003/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/008/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/010/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/009/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/015/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/011/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate063/012/
/gpfs/scratc

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/05/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/08/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/11/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/14/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/01/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/09/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/04/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/07/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/13/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/03/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/06/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/12/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230301_Plate016/10/
/gpfs/scratch1/shared/amf

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/003/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/008/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/010/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/009/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/015/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/011/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/016/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/012/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/004/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/017/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/013/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/005/
/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/20230518_Plate061/001/
/gpfs/scratc

,address_total,video_name,plate_nr,video_nr,date_imaged,fps,magnification,mode,parent_folder,data_table
306,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,001,069,1,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
62,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,001,060,1,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
82,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,001,056,1,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
16,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,001,063,1,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
136,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,001,049,1,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
...,...,...,...,...,...,...,...,...,...,...
206,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,27,048,27,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
227,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,28,048,28,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
218,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,29,048,29,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...
221,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,30,048,30,,20,50,BF,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...,/gpfs/scratch1/shared/amftrackflow/Hannah_set/...


In [64]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct, 
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/
/DATA/MYRISTATE/DATA/1_week/


In [65]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 60, upload_targ],
    dataframe,
    nr_parallel,
    "1:00:00",
    "flux_extract"
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1686255568686840909
307
Submitted batch job 2879209


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2879210


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2879215


...

Submitted batch job 2879229


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2879230


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2879231
